In [31]:
!pwd

/home/tomek/projects/detr-training


In [30]:
import argparse
import datetime
import json
import random
import time
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader, DistributedSampler

import datasets
import util.misc as utils
from datasets import build_dataset, get_coco_api_from_dataset
from engine import evaluate, train_one_epoch
from models import build_model

In [18]:
def get_args_parser():
    parser = argparse.ArgumentParser('Set transformer detector', add_help=False)
    parser.add_argument('--lr', default=1e-4, type=float)
    parser.add_argument('--lr_backbone', default=1e-5, type=float)
    parser.add_argument('--batch_size', default=2, type=int)
    parser.add_argument('--weight_decay', default=1e-4, type=float)
    parser.add_argument('--epochs', default=300, type=int)
    parser.add_argument('--lr_drop', default=200, type=int)
    parser.add_argument('--clip_max_norm', default=0.1, type=float,
                        help='gradient clipping max norm')

    # Model parameters
    parser.add_argument('--frozen_weights', type=str, default=None,
                        help="Path to the pretrained model. If set, only the mask head will be trained")
    # * Backbone
    parser.add_argument('--backbone', default='resnet50', type=str,
                        help="Name of the convolutional backbone to use")
    parser.add_argument('--dilation', action='store_true',
                        help="If true, we replace stride with dilation in the last convolutional block (DC5)")
    parser.add_argument('--position_embedding', default='sine', type=str, choices=('sine', 'learned'),
                        help="Type of positional embedding to use on top of the image features")

    # * Transformer
    parser.add_argument('--enc_layers', default=6, type=int,
                        help="Number of encoding layers in the transformer")
    parser.add_argument('--dec_layers', default=6, type=int,
                        help="Number of decoding layers in the transformer")
    parser.add_argument('--dim_feedforward', default=2048, type=int,
                        help="Intermediate size of the feedforward layers in the transformer blocks")
    parser.add_argument('--hidden_dim', default=256, type=int,
                        help="Size of the embeddings (dimension of the transformer)")
    parser.add_argument('--dropout', default=0.1, type=float,
                        help="Dropout applied in the transformer")
    parser.add_argument('--nheads', default=8, type=int,
                        help="Number of attention heads inside the transformer's attentions")
    parser.add_argument('--num_queries', default=100, type=int,
                        help="Number of query slots")
    parser.add_argument('--pre_norm', action='store_true')

    # * Segmentation
    parser.add_argument('--masks', action='store_true',
                        help="Train segmentation head if the flag is provided")

    # Loss
    parser.add_argument('--no_aux_loss', dest='aux_loss', action='store_false',
                        help="Disables auxiliary decoding losses (loss at each layer)")
    # * Matcher
    parser.add_argument('--set_cost_class', default=1, type=float,
                        help="Class coefficient in the matching cost")
    parser.add_argument('--set_cost_bbox', default=5, type=float,
                        help="L1 box coefficient in the matching cost")
    parser.add_argument('--set_cost_giou', default=2, type=float,
                        help="giou box coefficient in the matching cost")
    # * Loss coefficients
    parser.add_argument('--mask_loss_coef', default=1, type=float)
    parser.add_argument('--dice_loss_coef', default=1, type=float)
    parser.add_argument('--bbox_loss_coef', default=5, type=float)
    parser.add_argument('--giou_loss_coef', default=2, type=float)
    parser.add_argument('--eos_coef', default=0.1, type=float,
                        help="Relative classification weight of the no-object class")

    # dataset parameters
    parser.add_argument('--dataset_file', default='coco')
    parser.add_argument('--coco_path', type=str)
    parser.add_argument('--coco_panoptic_path', type=str)
    parser.add_argument('--remove_difficult', action='store_true')

    parser.add_argument('--output_dir', default='',
                        help='path where to save, empty for no saving')
    parser.add_argument('--device', default='cuda',
                        help='device to use for training / testing')
    parser.add_argument('--seed', default=42, type=int)
    parser.add_argument('--resume', default='', help='resume from checkpoint')
    parser.add_argument('--start_epoch', default=0, type=int, metavar='N',
                        help='start epoch')
    parser.add_argument('--eval', action='store_true')
    parser.add_argument('--num_workers', default=2, type=int)

    # distributed training parameters
    parser.add_argument('--world_size', default=1, type=int,
                        help='number of distributed processes')
    parser.add_argument('--dist_url', default='env://', help='url used to set up distributed training')
    return parser

import sys; sys.argv=['']
args = parser.parse_args()

In [26]:
args.coco_path = "/media/BIG2/COCO_DATASET/"

In [27]:
dataset_train = build_dataset(image_set='train', args=args)

loading annotations into memory...
Done (t=19.85s)
creating index...
index created!


In [29]:
dataset_train[1]

(tensor([[[-0.9020, -1.0562, -1.1247,  ..., -1.9809, -1.9809, -1.9809],
          [ 0.0056, -0.0458, -0.0801,  ..., -1.9809, -1.9809, -1.9809],
          [ 0.6221,  0.6563,  0.7077,  ..., -1.9809, -1.9809, -1.9809],
          ...,
          [ 0.2111,  0.2967,  0.2967,  ...,  0.5022,  0.8276,  1.1700],
          [ 0.2624,  0.3309,  0.2111,  ...,  0.2282,  0.5022,  0.8618],
          [ 0.2624,  0.3309,  0.2282,  ..., -0.0801,  0.0227,  0.2111]],
 
         [[-0.8277, -0.9678, -1.0203,  ..., -1.8782, -1.8782, -1.8782],
          [ 0.1176,  0.0651,  0.0476,  ..., -1.8782, -1.8782, -1.8782],
          [ 0.7479,  0.7829,  0.8354,  ..., -1.8782, -1.8782, -1.8782],
          ...,
          [-0.1975, -0.1275, -0.1275,  ...,  0.5728,  0.9405,  1.2381],
          [-0.1275, -0.0574, -0.1800,  ...,  0.3978,  0.6604,  0.8880],
          [-0.1275, -0.0574, -0.1625,  ...,  0.0651,  0.1527,  0.2227]],
 
         [[-0.4101, -0.5844, -0.6715,  ..., -1.7347, -1.7347, -1.7347],
          [ 0.4962,  0.4091,